# GROMACS tutorial for evaluating heat and mass transport properties of water adsorbed to 13X zeolite.

Instructions for running a GROMACS simulation to compute heat of adsorption and water diffusivity for zeolite-water. Provided by the Multi-Scale Modeling Lab of Politecnico di Torino (Italy). These resources are intended for pedagogical purposes, and were designed for the undergraduate and third-cycle courses at Politecnico di Torino. 

Matteo Fasano (matteo.fasano@polito.it), Michele Pellegrino (michele.pellegrino@polito.it)

All rights reserved (2025)

## Motivation

Potential for energy applications:
- Sorption energy storage allows to store heat for an indefinite time, with **no losses**;
- **Zeolite-water** couple leads to good energy density (about **100-500 kWh/m$^3$**);
- Zeolite is stable, safe and **inexpensive** (appx. 1€/L).

Open issues:
- Current zeolite batteries are underperforming respect to theoretical potential, due to a poor understanding of the involved **multiscale heat and mass transfer** phenomena;
- **Atomistic simulations** can be used to perform **fast and cheap** multiscale analysis of novel (and more performing) thermal batteries based on nanoporous materials.

Here, properties of zeolite 13X/water couple are studied.
- 2x2x2 Unit Cells of 13X Zeolite (Na$_{76}$Al$_{76}$Si$_{116}$O$_{384}$) 5216 atoms, 5x5x5 nm;
- Sodium ions are extra-framework cations (i.e. nonbonded) strongly influencing the adsorption proprieties of the framework.

Before we start, make sure all necessary Python modules are installed:

In [ ]:
import nglview as ng
import MDAnalysis as mda

import numpy as np
import matplotlib.pyplot as plt
import os

and that you moved to the tutorial's directory (`<your-local-path>/zeolite`):

In [ ]:
!pwd

As usual, we save the absolute work directory path:

In [ ]:
# Run this cell only once!
workdir = os.getcwd()
print("Work directory:",workdir)

## Input files

Extract and enter the topology folder:

In [ ]:
%cd {workdir}/top/

View the system geometry file by opening `13X76_2x2x2.gro` file:

In [ ]:
!cat 13X76_2x2x2.gro

These are the steps to create the topology file:
- Remove the Na atoms from the initial configuration file (e.g. by using `cat`, move all other atoms to a file named `13X76_2x2x2_noNa.gro`);
- Run `gmx x2top` to write the parameters for bonded and nonbonded interactions, taken from the given force field (in `13X_76.ff`);
- Add the Na atoms to the topology file, the charge needs to be chosen to ensure that the system is neutral;
- Add the force field parameters for water (shipped with GROMACS in `oplsaa.ff/tip4p.itp`).

The command to generate the topology should look something like this:

In [ ]:
!gmx x2top -f 13X76_2x2x2_noNa.gro -ff 13X_76 -name 13X76_2x2x2 -param -pbc -o 13X_temp.top

We already provide a topology file. The parameters of the zeolite matrix are moved to a different file (`zeolite.itp`), for the sake of readability:

In [ ]:
# Force field parameters for the zeolite matrix (no Na)
!head -n 200 zeolite.itp

In [ ]:
# Full topology (without water molecules)
!cat 13X.top

## 1 - Energy minimization

Extract and enter the energy minimization folder and have a look at the `.mdp` file:

In [ ]:
%cd {workdir}/1-EM

Have a look at the `.mdp` file:

In [ ]:
!cat em.mdp

Perform energy minimization of the dry structure by compiling the simulation. Note that GROMACS will throw warnings about position restraints: they are harmless and can be safely ignored by adding a `-maxwarn` flag.

In [ ]:
!gmx grompp -f em.mdp -n index.ndx -p ../top/13X.top -r ../top/restraints.gro -c ../top/13X76_2x2x2.gro -o 13X_dryem.tpr -maxwarn 2

Run the energy minimization simulation for the dry matrix:

In [ ]:
# Esc+o to collapse the output
!gmx mdrun -s *.tpr -o 13X_dryem.trr -c 13X_dryem.gro -g dryem.log -e dryem.edr -v

We can now have a look at the graphical representation of the initial setup. You can open the output `.gro` file `vmd` outside the notebook, instead of using `nglview` (`vmd` offers more visualization options).

In [ ]:
view_atoms = ng.show_structure_file("13X_dryem.gro")
view_atoms

The atomic representation is not suited to show the nanoporous structure of the zeolite matrix, let's switch to a surface visualization:

In [ ]:
view_surface = ng.show_structure_file("13X_dryem.gro")
view_surface.clear_representations()
view_surface.add_representation('surface', probeRadius=0.5, opacity=0.75)
view_surface

## 2 - Solvation

Extract and enter the zeolite hydration folder:

In [ ]:
%cd {workdir}/2-SOLV

Solvate the zeolite matrix by water for the low hydration level (80 water molecules):

In [ ]:
!gmx help insert-molecules

In [ ]:
!gmx insert-molecules -f ../1-EM/*.gro -ci tip4p.gro -o 13X_10.gro -radius 0.16 -nmol 80

and the high hydration level (2000 water molecules):

In [ ]:
!gmx insert-molecules -f ../1-EM/*.gro -ci tip4p.gro -o 13X_250.gro -nmol 2000

You can visualize the results with your favourite tool. If you want to use `nglviewer`, mind that the representation for water molecules need to be specified explicitly, for example:

In [ ]:
view_hydrated = ng.show_structure_file("13X_250.gro")
view_hydrated.clear_representations()
view_hydrated.add_representation('surface', probeRadius=0.5, opacity=0.75,selection="not water")
view_hydrated.add_representation('licorice',selection="water")
view_hydrated

## 3 - MD simulation of low hydration system

Perform energy minimization of the wet structure by the following sequence of commands:

In [ ]:
%cd {workdir}/3-MD10/3_1-EM

In [ ]:
!gmx grompp -f em.mdp -p ../../top/13X_10.top -r ../../top/restraints.gro -c ../../2-SOLV/13X_10.gro -o 13X_em.tpr -n ../index10.ndx -maxwarn 2

In [ ]:
# Esc+o to collapse
!gmx mdrun -s 13X_em.tpr -o 13X_em.trr -c 13X_em.gro -g em.log -e em.edr -v 

Perform thermal equilibration of the wet structure by the following sequence of commands:

In [ ]:
%cd {workdir}/3-MD10/3_2-TE

In [ ]:
!cat te.mdp

In [ ]:
!gmx grompp -f te.mdp -p ../../top/13X_10.top -r ../../top/restraints.gro -c ../3_1-EM/*.gro -o 13X_te.tpr -n ../index10.ndx -t ../3_1-EM/*.trr -maxwarn 2

In [ ]:
!gmx mdrun -s 13X_te.tpr -o 13X_te.trr -c 13X_te.gro -g te.log -e te.edr

Perform the final (production) run by the following sequence of commands: 

In [ ]:
%cd {workdir}/3-MD10/3_3-PROD

In [ ]:
!cat production.mdp

In [ ]:
!gmx grompp -f production.mdp -p ../../top/13X_10.top -r ../../top/restraints.gro -c ../3_2-TE/*.gro -o 13X_prod.tpr -n ../index10.ndx -t ../3_2-TE/*.trr -maxwarn 3

In [ ]:
!gmx mdrun -s 13X_prod.tpr -o 13X_prod.trr -c 13X_prod.gro -g prod.log -e prod.edr

You can check the simulation speed (ns/day) and the performance statistics at the end of the `.log` file:

In [ ]:
!tail -n 38 prod.log

Let's also visualize the trajectory using MDAnalysis and NGLView as for the CNT exercise:

In [ ]:
# If you have installed VMD
!vmd traj_comp.xtc 13X_prod.gro

In [ ]:
u_lowh = mda.Universe('13X_prod.tpr', 'traj_comp.xtc')
view_lowh = ng.show_mdanalysis(u_lowh)
view_lowh.clear_representations()
view_lowh.add_representation('surface', probeRadius=0.5, opacity=0.75,selection="not water")
view_lowh.add_representation('licorice',selection="water")
view_lowh

## 4 - MD simulation at high hydration

Same drill as for the low-hydration system: energy minimization, equilibration and production. 

In [ ]:
%cd {workdir}/4-MD250/4_1-EM

In [ ]:
!gmx grompp -f em.mdp -p ../../top/13X_250.top -r ../../top/restraints.gro -c ../../2-SOLV/13X_250.gro -o 13X_em.tpr -n ../index250.ndx -maxwarn 2

In [ ]:
!gmx mdrun -s 13X_em.tpr -o 13X_em.trr -c 13X_em.gro -g em.log -e em.edr -v

In [ ]:
%cd {workdir}/4-MD250/4_2-TE

In [ ]:
!gmx grompp -f te.mdp -p ../../top/13X_250.top -r ../../top/restraints.gro -c ../4_1-EM/*.gro -o 13X_te.tpr -n ../index250.ndx -t ../4_1-EM/*.trr -maxwarn 2

In [ ]:
!gmx mdrun -s 13X_te.tpr -o 13X_te.trr -c 13X_te.gro -g te.log -e te.edr

In [ ]:
%cd {workdir}/4-MD250/4_3-PROD

In [ ]:
!gmx grompp -f production.mdp -p ../../top/13X_250.top -r ../../top/restraints.gro -c ../4_2-TE/*.gro -o 13X_prod.tpr -n ../index250.ndx -t ../4_2-TE/*.trr -maxwarn 2

In [ ]:
!gmx mdrun -s 13X_prod.tpr -o 13X_prod.trr -c 13X_prod.gro -g prod.log -e prod.edr

You can compare the simulation speed with the low-hydration case:

In [ ]:
!tail -n 38 prod.log

...and again visualize the trajectory:

In [ ]:
u_highh = mda.Universe('13X_prod.tpr', 'traj_comp.xtc')
view_highh = ng.show_mdanalysis(u_highh)
view_highh.clear_representations()
view_highh.add_representation('surface', probeRadius=0.5, opacity=0.75,selection="not water")
view_highh.add_representation('licorice',selection="water")
view_highh

## 5 - Extracting observables and postprocessing

#### Heat of adsorption

Water-zeolite interactions are calculated from VdW and Coulomb potentials, between water and zeolite + Na ions. By increasing hydration level, the specific water-zeolite interactions are reduced (thus also heat of adsorption decreases).

The heat of adsorption can be estimated by the difference between the water-zeolite and the water-water specific interaction energies:

$$-q=\Delta H\simeq U_{WZ}-U_{WW}$$

Post-process final trajectories in order to obtain the energy of the system:  

In [ ]:
%cd {workdir}/postprocessing

In [ ]:
!cat ENERGY/inputE.txt

In [ ]:
!gmx energy -f ../3-MD10/3_3-PROD/*edr -o ./ENERGY/Energy_10.xvg < ./ENERGY/inputE.txt

In [ ]:
!gmx energy -f ../4-MD250/4_3-PROD/*edr -o ./ENERGY/Energy_250.xvg < ./ENERGY/inputE.txt

Let's have a look at the output of `gmx energy`:

In [ ]:
!cat ENERGY/Energy_250.xvg

In [ ]:
# If you have installed Grace
!xmgrace -free -nxy ENERGY/Energy_250.xvg

You can read the `.xvg` file as a text file directly with `numpy` (or `pandas`, if you are so inclined):

In [ ]:
energies_10 = np.loadtxt("ENERGY/Energy_10.xvg", comments=("@", "#"))
energies_250 = np.loadtxt("ENERGY/Energy_250.xvg", comments=("@", "#"))

# The output should be now stored in a 2D array: #time steps x (time,variable1,variable2,...):
print(energies_10.shape)

In [ ]:
# Analysis and plotting...
n_wat_10 = 80
n_wat_250 = 2000
u_WN_10 = (energies_10[:,9]+energies_10[:,10])/n_wat_10
u_WN_250 = (energies_250[:,9]+energies_250[:,10])/n_wat_250
u_WW_10 = (energies_10[:,7]+energies_10[:,8])/n_wat_10
u_WW_250 = (energies_250[:,7]+energies_250[:,8])/n_wat_250
u_WZ_10 = (energies_10[:,5]+energies_10[:,6])/n_wat_10
u_WZ_250 = (energies_250[:,5]+energies_250[:,6])/n_wat_250

In [ ]:
# TODO: calculate the heat of adsorption
q_10 = u_WW_10-(u_WZ_10+u_WN_10)
q_250 = u_WW_250-(u_WZ_250+u_WN_250)

In [ ]:
# TODO: add a plot for the heat of adsorption
fig1, ax1 = plt.subplots()
plt.plot(energies_10[:,0],u_WW_10,'ro',label='U_WW, low hydration')
plt.plot(energies_250[:,0],u_WW_250,'bo',label='U_WW, high hydration')
plt.plot(energies_10[:,0],u_WZ_10+u_WN_10,'rx',label='U_WZ, low hydration')
plt.plot(energies_250[:,0],u_WZ_250+u_WN_250,'bx',label='U_WZ, high hydration')
plt.plot(energies_10[:,0],q_10,'r-',label='q_WZ, low hydration')
plt.plot(energies_250[:,0],q_250,'b-',label='q_WZ, high hydration')
plt.xlabel('time [ps]')
plt.ylabel('specific energy [kJ/mol]')
plt.legend()

#### Self-diffusivity of water

The self-diffusion coefficient $D$ of the water molecules is determined following the classical relationship of Einstein and computing the mean square displacement (MSD) as:

$$\mbox{MSD}(t) = <||\vec{r}(t)-\vec{r}(0)||^2> \sim 6Dt$$

for $t\rightarrow\infty$.

Nanoconfinement reduces water mobility, namely self-diffusion coefficient. Higher hydration levels also modify the water mobility within the zeolite structure.

Post-process final trajectories in order to obtain the mean square displacement of water molecules in the system: 

In [ ]:
!mkdir MSD

In [ ]:
!echo SOL | gmx msd -f ../3-MD10/3_3-PROD/*xtc -s ../3-MD10/3_3-PROD/*tpr -o ./MSD/msd_10.xvg

In [ ]:
!echo SOL | gmx msd -f ../4-MD250/4_3-PROD/*xtc -s ../4-MD250/4_3-PROD/*tpr -o ./MSD/msd_250.xvg

Let's have a look at the output of `gmx msd`:

In [ ]:
!cat MSD/msd_10.xvg
# !cat MSD/msd_250.xvg

In [ ]:
# If you have Grace installed
!xmgrace -free MSD/msd_10.xvg
# !xmgrace -free MSD/msd_250.xvg

In [ ]:
msd_10 = np.loadtxt("MSD/msd_10.xvg", comments=("@", "#"))
msd_250 = np.loadtxt("MSD/msd_250.xvg", comments=("@", "#"))

# Now the output should be stored in a 2D array: #time steps x (time, MSD)
print(msd_10.shape)

In [ ]:
coeff_10 = np.polyfit(msd_10[:,0],msd_10[:,1],deg=1)
coeff_250 = np.polyfit(msd_250[:,0],msd_250[:,1],deg=1)
# Remember to convert from nm^2/ps to cm^2/s
D_10 = (1e-2)*coeff_10[0]/6
D_250 = (1e-2)*coeff_250[0]/6
print("D (low hydration) =",D_10,"cm^2/s")
print("D (high hydration) =",D_250,"cm^2/s")

In [ ]:
# TODO: Plot the linear fit of the MSD equation
fig2, ax2 = plt.subplots()
plt.plot(msd_10[:,0],msd_10[:,1],'ro',label='Low hydration')
plt.plot(msd_250[:,0],msd_250[:,1],'bs',label='High hydration')
plt.plot(msd_10[:,0],np.polyval(coeff_10,msd_10[:,0]),'r-')
plt.plot(msd_250[:,0],np.polyval(coeff_250,msd_250[:,0]),'b-')
plt.xlabel("time [ps]")
plt.ylabel("MSD [nm^2]")
plt.legend()